
#  **The difference between :**
**'sparse_categorical_crossentropy'** and
**'categorical_crossentropy'**
(With an example)





#  **Before Starting:**

To Load The Data (that I use in this demonstration) :

See the first cells of the Notebook: **week 4** of the course 2 : **Convolutional Neural Networks in TensorFlow**

of **DeepLearning.AI TensorFlow Developer Professional Certificate**. here the link ot the course: https://www.coursera.org/learn/convolutional-neural-networks-tensorflow


> **Description**








> From **First Step** to **Third Step**:


I do a series of treatments on image files,(+checking the convolutional model: **compiling it with different** parameters values to show the difference between the two values of **loss parameter** : 

**'sparse_categorical_crossentropy'**  and **'categorical_crossentropy'** 

This demonstration **doesn't use** the **image generators**(you will understand why if you follow the 3 different steps)



> ***Just in passing:*** 

If you try to train the model of the course with **train_generator** and **validation_generator**(but not with the value of the course, but by setting the value of **loss = 'sparse_categorical_crossentropy'**,(instead of **'categorical_crossentropy'**: 

this gives an error (you can try it):

**The message of the error:**

**'InvalidArgumentError:** logits and labels must have the same first dimension, got logits shape [126,3] and labels shape [378]' 

So, it only works with **loss ='categorical_crossentropy '**): because in our case: it is **the generator** that **labels** all the images fotr us(...)


> Does this mean that **'sparse_categorical_crossentropy'** never works with **generators**?


**Loud No:** the image generaors can work with **sparse_categorical_crossentropy'**, only in this case:

When  **y_train**, or **y_test** (the vectors of our labels) are not included in **train_generator**, and **validation_generator**) successively, that's means in the case where there is **separation** between : **X_train & y_train** (of the train dataset ), and **X_test & y_test** (of the test dataset): so this doesn't prevent the application of  **generators** (on them...) and we **must** in this case use **'sparse_categorical_crossentropy'**(if -of course-  y_train and y_test haven't already been reshaped (by us)...(**see the demonstration, below  to figure out it with a pracical example)**


#  **First_Step**:  
we create (**X_train, y_train)**, **(X_test,y_test)** from the image files **without using generators** in order to transform them in numpy arrays

**Note:** generatos do this work for us , but we need **X_train** (features) and **y_train** (labels) separated (for the rest...)





> **I-1 Creating (X_train, y_train)**



In [ ]:
#Note: 
#I already did zip_ref.extractall('/content') and not on /tmp, that's why you will see /content instead of /tmp 

rock_dir = os.path.join('/content/rps/rock')
paper_dir = os.path.join('/content/rps/paper')
scissors_dir = os.path.join('/content/rps/scissors')


In [ ]:
from matplotlib.image import imread
import numpy as np
import os

paper_dir = os.path.join('/content/rps/paper')
rock_dir = os.path.join('/content/rps/rock')
scissors_dir = os.path.join('/content/rps/scissors')


list_path_images_train_paper= [os.path.join(paper_dir,fn) for fn in os.listdir(paper_dir)]
list_path_images_train_rock= [os.path.join(rock_dir,fn) for fn in os.listdir(rock_dir)]
list_path_images_train_scissors = [os.path.join(scissors_dir,fn) for fn in os.listdir(scissors_dir)]

#We transform the images in numpy arrays
train_paper = [imread(path_image) for path_image in list_path_images_train_paper]
train_paper= np.array(train_paper)

# in order to get the shape (840, 300, 300, 3)
train_paper= train_paper[:,:,:,:3]

#We define the labels
#Note: with the generator version if we do : 
#train_generator.class_indices, it gives us : {'paper': 0, 'rock': 1, 'scissors': 2} (So we keep this order)
y_train_paper = np.zeros(840)

#we repeat the same treatment with rock subdirectory
train_rock = [imread(path_image) for path_image in list_path_images_train_rock]
train_rock= np.array(train_rock)
# in order to get the shape (840, 300, 300, 3)
train_rock= train_rock[:,:,:,:3]
#we define the labels
y_train_rock = np.ones(840)

#we repeat the same treatment with scissors subdirectory
train_scissors = [imread(path_image) for path_image in list_path_images_train_scissors]
train_scissors= np.array(train_scissors)
#in order to get the shape (840, 300, 300, 3)
train_scissors= train_scissors[:,:,:,:3]
#we define the labels
y_train_scissors = (np.ones(840))*2

X_train= np.concatenate((train_paper,train_rock,train_scissors))
y_train= np.concatenate((y_train_paper,y_train_rock,y_train_scissors))

In [ ]:
#just a little check
X_train.shape


(2520, 300, 300, 3)

In [ ]:
#We have two separated numpy arrays X_train and y_train with the same length (but different shapes).
#The elements are not shuffled (we have the elments of class 0 (paper))at the beginning, then come the elements
#of class 1 (rock), then the elements of class 2 (scissors)
#To solve this, we want to shuffle each of them, such that corresponding elements continue to correspond.
#we define this function:

def shuffle_same_order(X, y):
    assert len(X) == len(y)
    shuffled_X = np.empty(X.shape, dtype=X.dtype)
    shuffled_y = np.empty(y.shape, dtype=y.dtype)
    permutation = np.random.permutation(len(X))
    for old_index, new_index in enumerate(permutation):
        shuffled_X[new_index] = X[old_index]
        shuffled_y[new_index] = y[old_index]
    return shuffled_X, shuffled_y

#Note: assert statement has a condition or expression which is supposed to be always true. 
#If the condition is false assert halts the program and gives an AssertionError.

In [ ]:
(X_train,y_train) = shuffle_same_order(X_train, y_train)



> **I-2 Creating (X_test, y_test)**



In [ ]:
#To build (X_test and y_test), we do the same treatment for test_datset (validation dateset)

paper_dir_test = os.path.join('/content/rps-test-set/paper')
rock_dir_test = os.path.join('/content/rps-test-set/rock')
scissors_dir_test = os.path.join('/content/rps-test-set/scissors')

list_path_images_test_paper= [os.path.join(paper_dir_test,fn) for fn in os.listdir(paper_dir_test)]
list_path_images_test_rock= [os.path.join(rock_dir_test,fn) for fn in os.listdir(rock_dir_test)]
list_path_images_test_scissors = [os.path.join(scissors_dir_test,fn) for fn in os.listdir(scissors_dir_test)]

#We transform the images in numpy arrays
test_paper = [imread(path_image) for path_image in list_path_images_test_paper]
test_paper= np.array(test_paper)

#in order to get the shape (124, 300, 300, 3)
test_paper= test_paper[:,:,:,:3]
#we define the labels
y_test_paper = np.zeros(124)

#we repeat the same treatment with rock subdirectory
test_rock = [imread(path_image) for path_image in list_path_images_test_rock]
test_rock= np.array(test_rock)
# to get the shape (124, 300, 300, 3)
test_rock= test_rock[:,:,:,:3]
#we define the labels
y_test_rock = np.ones(124)

#we repeat the same treatment with scissors subdirectory
test_scissors = [imread(path_image) for path_image in list_path_images_test_scissors]
test_scissors= np.array(test_scissors)
# to get the shape (124, 300, 300, 3)
test_scissors= test_scissors[:,:,:,:3]
#we define the labels
y_test_scissors = (np.ones(124))*2

X_test= np.concatenate((test_paper,test_rock,test_scissors))
y_test= np.concatenate((y_test_paper,y_test_rock,y_test_scissors))

In [ ]:
(X_test,y_test) = shuffle_same_order(X_test, y_test)



#  **Second Step:  Creating the model**



In [ ]:
import tensorflow as tf
#create our model (the same that we created for training the train_generator), but we reduce the filters
#for convolutions because, we have the original shape (300,300,3), not (150,150,3)

model = tf.keras.models.Sequential([
    
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])





> I- We will compile the model with     **'categorical_crossentropy'**, without any modification in labels vectors shape: y_train and y_test

we will end up with an **error** when we will **train** the model ! (**see the error when training the model in Third step)**

In [ ]:
#we will compile our model with loss ='categorical_crossentropy'

model.compile(loss='categorical_crossentropy', optimizer= 'rmsprop', metrics=['accuracy'])



#  **Third Step: Problem and solutions:**





In [ ]:
#below, when we train the model, we end up with an error 
#because the shape of y_train and y_test are vectors like this:
# y_train = array([0., 2., 0., ..., 1., 0., 2.])
# but the shape should become like this:
# array([[1., 0., 0.],
#        [0., 1., 0.],
#        [1., 0., 0.],
#         ...,
#        [0., 0., 1.],
#        [0., 0., 1.],
#        [0., 1., 0.]], dtype=float32)
#in order to distinguish classes (categorical values) from continuous values 

# To solve this problem, we have two solutions :

# ---Solution_1:  we reshape y_train and y_test as mentioned above.
# ---Solution_2:  we remplace 'categorical_crossentropy' with'sparse_categorical_crossentropy


In [ ]:
#The error we get:
model.fit(X_train, y_train, epochs=25, validation_data=(X_test,y_test))

Epoch 1/25


ValueError: ignored




> **III-1 Solution_1**: 

we **reshape** **y_train** and **y_test** as mentioned above & but we keep the same parameter: **'categorical_crossentropy'**



In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
y_train

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

In [ ]:
y_test

array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]], dtype=float32)

In [ ]:
#Note, we shouldn't use the model that we used above (I mean the one we  already trained above 
#and gives us an  error) # : we should run and re-compile the model a 2nd time then we train it  as following:

#Note: 
#we are not trying here to see if the accuracy is good or not
#The goal, here is just to figure out when we can use 
#'sparse_categorical_crossentropy' or 'categorical_crossentropy 

In [ ]:
model.fit(X_train, y_train, epochs=20, validation_data=(X_test,y_test))

Epoch 1/20
79/79 [==============================] - 219s 3s/step - loss: 2.5304 - accuracy: 0.5879 - val_loss: 1.7807 - val_accuracy: 0.7231
Epoch 2/20
79/79 [==============================] - 218s 3s/step - loss: 0.1382 - accuracy: 0.9833 - val_loss: 2.1031 - val_accuracy: 0.7634
Epoch 3/20
79/79 [==============================] - 218s 3s/step - loss: 0.2231 - accuracy: 0.9965 - val_loss: 1.2695 - val_accuracy: 0.7957
Epoch 4/20
79/79 [==============================] - 217s 3s/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 1.8601 - val_accuracy: 0.7796
Epoch 5/20
79/79 [==============================] - 217s 3s/step - loss: 1.6458 - accuracy: 0.9556 - val_loss: 1.5597 - val_accuracy: 0.8038
Epoch 6/20
79/79 [==============================] - 216s 3s/step - loss: 0.2821 - accuracy: 0.9931 - val_loss: 1.6363 - val_accuracy: 0.7661
Epoch 7/20
79/79 [==============================] - 216s 3s/step - loss: 6.1551e-05 - accuracy: 1.0000 - val_loss: 2.2484 - val_accuracy: 0.7688
Epoch 8/2



> **III-2 Solution 2**:

 we just replace **'categorical_crossentropy'** by **'sparse_categorical_crossentropy'** (and **keeping the original shape of y_train and y_test**)

==> So the difference between **solution 1** and **solution 2**: that in this solution 2 :  **'sparse_categorical_crossentropy'** does the work that we already did in **reshaping** : y_train & y_test, in solution 1 (here it does automatically, the work for us) 


In [ ]:
# First : we (re)run the code cell of our model (again)
#and we use the code (below) for compiling it (So with 'Sparse_categorical_crossentropy' and not 'categorical_crossentropy')
#for this we sohould use the original forms of y_train and y_test (before reshaping them)
#I mean this original shape array([0., 2., 0., ..., 1., 0., 2.])
#So I just run the code cells (of step1 ), that gives us the origial y_train and y_test 


In [ ]:
# And now, Just for checking (after running the code cells of step 1):
y_train

array([0., 1., 0., ..., 1., 2., 0.])

In [ ]:
#The most important thing here: thar we  compile our model with loss ='sparse_categorical_crossentropy'
#and we will see: we won't end up with an error while training the model

model.compile(loss='sparse_categorical_crossentropy', optimizer= 'rmsprop', metrics=['accuracy'])

In [ ]:
#Note, I set epochs =5 (to make the simulation faster)
#Because the goal here is'nt to check the performance of the model 
#But the use of 'categorical_crossentropy', and 'sparse_categorical_crossentropy'

model.fit(X_train, y_train, epochs=5, validation_data=(X_test,y_test))

Epoch 1/5
79/79 [==============================] - 218s 3s/step - loss: 2.0025 - accuracy: 0.5474 - val_loss: 0.7967 - val_accuracy: 0.6129
Epoch 2/5
79/79 [==============================] - 218s 3s/step - loss: 0.0998 - accuracy: 0.9711 - val_loss: 0.4762 - val_accuracy: 0.8522
Epoch 3/5
79/79 [==============================] - 216s 3s/step - loss: 0.0982 - accuracy: 0.9842 - val_loss: 0.8167 - val_accuracy: 0.7849
Epoch 4/5
79/79 [==============================] - 218s 3s/step - loss: 0.0290 - accuracy: 0.9964 - val_loss: 0.5804 - val_accuracy: 0.7796
Epoch 5/5
79/79 [==============================] - 217s 3s/step - loss: 7.5096e-04 - accuracy: 1.0000 - val_loss: 1.0288 - val_accuracy: 0.8011
